In [1]:
#import dependancies
import json
from elasticsearch import Elasticsearch, helpers

In [2]:
#initialize Elasticsearch client
client = Elasticsearch()

In [3]:
#query the idex called "pandemics".
client.search(index='pandemics', filter_path=['hits.hits._id', 'hits.hits._type'])

/usr/local/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'hits': {'hits': [{'_type': '_doc', '_id': 'MB0Z_H4BR8yRQYoutRND'},
   {'_type': '_doc', '_id': 'MR0Z_H4BR8yRQYoutRND'},
   {'_type': '_doc', '_id': 'Mh0Z_H4BR8yRQYoutRND'},
   {'_type': '_doc', '_id': 'Mx0Z_H4BR8yRQYoutRND'},
   {'_type': '_doc', '_id': 'NB0Z_H4BR8yRQYoutRND'},
   {'_type': '_doc', '_id': 'NR0Z_H4BR8yRQYoutRND'},
   {'_type': '_doc', '_id': 'Nh0Z_H4BR8yRQYoutRND'},
   {'_type': '_doc', '_id': 'Nx0Z_H4BR8yRQYoutRND'},
   {'_type': '_doc', '_id': 'OB0Z_H4BR8yRQYoutRND'},
   {'_type': '_doc', '_id': 'OR0Z_H4BR8yRQYoutRND'}]}}

### Get 10 articles

In [4]:
res = client.search(index="pandemics", query= {"match_all": {}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(article_title)s %(section_title)s" % hit["_source"])

Got 802 Hits:
Pandemic Summary
Pandemic Definition
Pandemic Stages
Pandemic Severity
Pandemic Management
Pandemic HIV/AIDS
Pandemic COVID-19
Pandemic Notable outbreaks
Pandemic Cholera
Pandemic Influenza


### Create a simple (match) query and extract the results from the nested JSON response.

In [21]:
question = "vaccine"
query = {"match": {"text": question}}

res = client.search(index="pandemics", query=query, size=50)
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(article_title)s %(section_title)s" % hit["_source"])

Got 76 Hits:
HIV/AIDS Vaccination
HIV/AIDS Vaccination
Science diplomacy and pandemics See also
Science diplomacy and pandemics See also
COVID-19 pandemic Vaccine
COVID-19 pandemic Vaccine
Targeted immunization strategies See also
Targeted immunization strategies See also
Virus Vaccines
Virus Vaccines
Pandemic prevention Vaccination
Pandemic prevention Vaccination
Cholera Vaccination
Cholera Vaccination
Targeted immunization strategies Background
Targeted immunization strategies Background
Science diplomacy and pandemics Ebola
Science diplomacy and pandemics Ebola
HIV/AIDS Treatment
HIV/AIDS Treatment
COVID-19 pandemic 2020
COVID-19 pandemic 2020
COVID-19 pandemic United Kingdom
COVID-19 pandemic United Kingdom
Science diplomacy and pandemics Severe acute respiratory syndrome (SARS)
Science diplomacy and pandemics Severe acute respiratory syndrome (SARS)
Swine influenza 1976 U.S.
Swine influenza 1976 U.S.
Virus Weapons
Virus Weapons
Pandemic Measles
Pandemic Measles
Superspreading even